In [2]:
import pandas as pd
import numpy as np

In [3]:
data='https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv'

In [4]:
!wget $data

--2025-10-13 17:19:58--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80876 (79K) [text/plain]
Saving to: ‘course_lead_scoring.csv.1’

course_lead_scoring 100%[===================>]  78.98K  --.-KB/s    in 0.008s  

2025-10-13 17:19:59 (9.35 MB/s) - ‘course_lead_scoring.csv.1’ saved [80876/80876]



In [5]:
df=pd.read_csv('course_lead_scoring.csv')

In [6]:
df.head(5)

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [7]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [8]:
#no of null values in each column
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [9]:
strings = list(df.dtypes[df.dtypes == 'object'].index)
strings

['lead_source', 'industry', 'employment_status', 'location']

In [10]:
numeric_cols = df.select_dtypes(include=['number']).columns
numeric_cols

Index(['number_of_courses_viewed', 'annual_income', 'interaction_count',
       'lead_score', 'converted'],
      dtype='object')

In [11]:
df[strings] = df[strings].fillna('NA')
df[numeric_cols]=df[numeric_cols].fillna(0.0)

In [12]:
#Unique values in each column and number of unique values
for col in df.columns:
    print(col)
    print(df[col].unique()[:5])
    print(df[col].nunique())
    print()

lead_source
['paid_ads' 'social_media' 'events' 'referral' 'organic_search']
6

industry
['NA' 'retail' 'healthcare' 'education' 'manufacturing']
8

number_of_courses_viewed
[1 5 2 3 0]
10

annual_income
[79450. 46992. 78796. 83843. 85012.]
1268

employment_status
['unemployed' 'employed' 'NA' 'self_employed' 'student']
5

location
['south_america' 'australia' 'europe' 'africa' 'middle_east']
8

interaction_count
[4 1 3 6 2]
12

lead_score
[0.94 0.8  0.69 0.87 0.62]
101

converted
[1 0]
2



In [13]:
#Question 1
# What is the most frequent observation (mode) for the column industry?
#NA
#technology
#healthcare 
#retail

In [14]:
# Get the mode (most frequent value) of 'industry'
most_frequent_industry = df['industry'].mode()[0]

print("Most frequent industry:", most_frequent_industry)

Most frequent industry: retail


In [15]:
#Question 2
#What are the two features that have the biggest correlation?

#interaction_count and lead_score
#number_of_courses_viewed and lead_score
#number_of_courses_viewed and interaction_count
#annual_income and interaction_count

In [16]:
# Identify numeric columns
numeric_cols = df.select_dtypes(include=['number']).columns

# Specify your target variable name (example: 'converted')
target_col = 'converted'

# Exclude the target variable
numeric_features = [col for col in numeric_cols if col != target_col]

# Compute the correlation matrix for numeric features only
correlation_matrix = df[numeric_features].corr()

# Display
print(correlation_matrix)



                          number_of_courses_viewed  annual_income  \
number_of_courses_viewed                  1.000000       0.009770   
annual_income                             0.009770       1.000000   
interaction_count                        -0.023565       0.027036   
lead_score                               -0.004879       0.015610   

                          interaction_count  lead_score  
number_of_courses_viewed          -0.023565   -0.004879  
annual_income                      0.027036    0.015610  
interaction_count                  1.000000    0.009888  
lead_score                         0.009888    1.000000  


In [17]:
# Find the two features with the largest absolute correlation (ignoring the diagonal)
corr_matrix_abs = correlation_matrix.abs()

top_corr = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool)) \
    .stack() \
    .sort_values(ascending=False)

In [18]:
print(top_corr[:1])

annual_income  interaction_count    0.027036
dtype: float64


In [19]:
# ### Split the data

# * Split your data in train/val/test sets with 60%/20%/20% distribution.

# * Use Scikit-Learn for that (the `train_test_split` function) and set the seed to `42`

from sklearn.model_selection import train_test_split
RANDOM_SEED = 42


df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=RANDOM_SEED)
df_train, df_val = train_test_split(df_full_train, test_size=0.25,
                                    random_state=RANDOM_SEED)
# * Make sure that the target value `y` is not in your dataframe.
len(df_train), len(df_val), len(df_test)

# Resetting indices
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

# Setting targets
y_train = df_train[target_col].values
y_val = df_val[target_col].values
y_test = df_test[target_col].values

del df_train[target_col], df_val[target_col], df_test[target_col]

In [ ]:
#Question 3
#Calculate the mutual information score between y and other categorical variables in the dataset. Use the training set only.
#Round the scores to 2 decimals using round(score, 2).import pandas as pd

In [21]:
from sklearn.feature_selection import mutual_info_classif

# Select categorical columns from training data
categorical_cols = df_train.select_dtypes(include=['object', 'category']).columns

# Convert categorical variables to numeric (factorize for MI)
X_train_encoded = df_train[categorical_cols].apply(lambda col: col.factorize()[0])

# Compute mutual information between each categorical feature and target
mi_scores = mutual_info_classif(X_train_encoded, y_train, discrete_features=True, random_state=42)

# Create a pandas Series with feature names and MI scores
mi_scores = pd.Series(mi_scores, index=categorical_cols).sort_values(ascending=False).round(2)

# Display the results
print("Mutual Information Scores (Training Set, Categorical Features):")
print(mi_scores)


Mutual Information Scores (Training Set, Categorical Features):
lead_source          0.04
employment_status    0.01
industry             0.01
location             0.00
dtype: float64


In [22]:
# `lead_source` has the highest mutual information score.

In [23]:
#Question 4
#Now let's train a logistic regression.
#Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.
#Fit the model on the training dataset.
#To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
#model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
#Calculate the accuracy on the validation dataset and round it to 2 decimal digits.
#What accuracy did you get?
#0.64
#0.74
#0.84
#0.94


In [32]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score

# 1. Identify categorical and numerical columns
categorical_cols = df_train.select_dtypes(include=['object', 'category']).columns
numerical_cols = df_train.select_dtypes(include=['number']).columns

# 2. One-hot encode categorical variables
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

X_train_encoded = pd.DataFrame(
    encoder.fit_transform(df_train[categorical_cols]),
    columns=encoder.get_feature_names_out(categorical_cols)
)

X_val_encoded = pd.DataFrame(
    encoder.transform(df_val[categorical_cols]),
    columns=encoder.get_feature_names_out(categorical_cols)
)

# 3. Combine numeric and encoded categorical columns
X_train_final = pd.concat([df_train[numerical_cols].reset_index(drop=True), X_train_encoded.reset_index(drop=True)], axis=1)
X_val_final = pd.concat([df_val[numerical_cols].reset_index(drop=True), X_val_encoded.reset_index(drop=True)], axis=1)

# 4. Train logistic regression model
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train_final, y_train)

# 5. Predict and calculate accuracy on validation set
y_pred = model.predict(X_val_final)
accuracy = accuracy_score(y_val, y_pred)

# 6. Round accuracy to 2 decimals
print(f"Validation Accuracy: {round(accuracy, 2)}")

Validation Accuracy: 0.7


In [26]:
#Accuracy : 0.74

In [ ]:
#Question 5
#Let's find the least useful feature using the feature elimination technique.
#Train a model using the same features and parameters as in Q4 (without rounding).
#Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
#For each feature, calculate the difference between the original accuracy and the accuracy without the feature.
#Which of following feature has the smallest difference?
#'industry'
#'employment_status'
#'lead_score'

In [27]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score

def prepare_data(train_df, val_df, drop_feature=None):
    # Drop a specific feature if specified
    if drop_feature:
        train_df = train_df.drop(columns=[drop_feature])
        val_df = val_df.drop(columns=[drop_feature])
    
    categorical_cols = train_df.select_dtypes(include=['object', 'category']).columns
    numerical_cols = train_df.select_dtypes(include=['number']).columns
    
    # One-hot encode categorical variables
    encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    X_train_encoded = pd.DataFrame(
        encoder.fit_transform(train_df[categorical_cols]),
        columns=encoder.get_feature_names_out(categorical_cols)
    )
    X_val_encoded = pd.DataFrame(
        encoder.transform(val_df[categorical_cols]),
        columns=encoder.get_feature_names_out(categorical_cols)
    )
    
    # Combine numerical and encoded categorical columns
    X_train_final = pd.concat([train_df[numerical_cols].reset_index(drop=True),
                               X_train_encoded.reset_index(drop=True)], axis=1)
    X_val_final = pd.concat([val_df[numerical_cols].reset_index(drop=True),
                             X_val_encoded.reset_index(drop=True)], axis=1)
    return X_train_final, X_val_final

# 1. Baseline model (all features)
X_train_full, X_val_full = prepare_data(df_train, df_val)
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train_full, y_train)
baseline_accuracy = accuracy_score(y_val, model.predict(X_val_full))
print("Baseline accuracy:", baseline_accuracy)

# 2. Feature elimination
features_to_test = ['industry', 'employment_status', 'lead_score']
diffs = {}

for feature in features_to_test:
    X_train_sub, X_val_sub = prepare_data(df_train, df_val, drop_feature=feature)
    model.fit(X_train_sub, y_train)
    acc = accuracy_score(y_val, model.predict(X_val_sub))
    diffs[feature] = baseline_accuracy - acc

# 3. Show results
print("\nAccuracy differences after removing each feature:")
for f, d in diffs.items():
    print(f"{f}: {d:.4f}")

# Find least useful feature
least_useful = min(diffs, key=diffs.get)
print(f"\nLeast useful feature: {least_useful}")


Baseline accuracy: 0.6996587030716723

Accuracy differences after removing each feature:
industry: 0.0000
employment_status: 0.0034
lead_score: -0.0068

Least useful feature: lead_score


In [ ]:
#Now let's train a regularized logistic regression.
#Let's try the following values of the parameter C: [0.01, 0.1, 1, 10, 100].
#Train models using all the features as in Q4.
#Calculate the accuracy on the validation dataset and round it to 3 decimal digits.
#Which of these C leads to the best accuracy on the validation set?

#0.01
#0.1
#1
#10
#100


In [33]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np

# Reuse the encoded and combined training/validation data from Q4
# (X_train_final, X_val_final, y_train, y_val must already be defined)

# List of C values to test
C_values = [0.01, 0.1, 1, 10, 100]
results = []

for C in C_values:
    model = LogisticRegression(
        solver='liblinear',
        penalty='l2',
        C=C,
        max_iter=1000,
        random_state=42
    )
    model.fit(X_train_final, y_train)
    y_pred = model.predict(X_val_final)
    acc = accuracy_score(y_val, y_pred)
    results.append((C, round(acc, 3)))

# Display results
print("Validation Accuracies (rounded to 3 decimals):")
for C, acc in results:
    print(f"C={C:<6}  Accuracy={acc:.3f}")

# Pick the best (ties → smaller C)
best_acc = max(a for _, a in results)
best_candidates = [C for C, a in results if a == best_acc]
best_C = min(best_candidates)

print(f"\nBest C: {best_C} with validation accuracy {best_acc:.3f}")


Validation Accuracies (rounded to 3 decimals):
C=0.01    Accuracy=0.700
C=0.1     Accuracy=0.700
C=1       Accuracy=0.700
C=10      Accuracy=0.700
C=100     Accuracy=0.700

Best C: 0.01 with validation accuracy 0.700
